In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import numpy as np
import string

Using TensorFlow backend.


# Data Preparation

Load text

In [2]:
file = open("DATASETS/livros.txt")
doc = file.read()

Clean text

In [3]:
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [4]:
tokens = clean_doc(doc)

In [5]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

Total Tokens: 109609
Unique Tokens: 15144


Make sequences

In [6]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    #print(len(line))
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 109558


In [7]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
out_filename = 'model/livros_sequences.txt'
save_doc(sequences,out_filename)

Encode Sequences

In [8]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

Sequence Inputs and Output

In [9]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

# Train Language Model

In [10]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=length-1))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            757250    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 15145)             1529645   
Total params: 2,437,795
Trainable params: 2,437,795
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/100
109558/109558 [==============================] - 182s 2ms/step - loss: 7.3179 - acc: 0.0401
Epoch 2/100
109558/109558 [==============================] - 177s 2ms/step - loss: 6.8388 - acc: 0.0527
Epoch 3/100
109558/109558 [==============================] - 185s 2ms/step - loss: 6.6747 - acc: 0.0591
Epoch 4/100
109558/109558 [==============================] - 180s 2ms/step - loss: 6.5754 - acc: 0.0622
Epoch 5/100
109558/109558 [==============================] - 182s 2ms/step - loss: 6.4789 - acc: 0.0670
Epoch 6/100
109558/109558 [==============================] - 183s 2ms/step - loss: 6.3242 - acc: 0.0747
Epoch 7/100
109558/109558 [==============================] - 189s 2ms/step - loss: 6.1492 - acc: 0.0815
Epoch 8/100
109558/109558 [==============================] - 188s 2ms/step - loss: 6.0044 - acc: 0.0909
Epoch 9/100
109558/109558 [==============================] - 191s 2ms/step - loss: 5.8798 - acc

109558/109558 [==============================] - 197s 2ms/step - loss: 2.0146 - acc: 0.5401
Epoch 77/100
109558/109558 [==============================] - 201s 2ms/step - loss: 1.9873 - acc: 0.5443
Epoch 78/100
109558/109558 [==============================] - 191s 2ms/step - loss: 1.9637 - acc: 0.5494
Epoch 79/100
109558/109558 [==============================] - 200s 2ms/step - loss: 1.9366 - acc: 0.5545
Epoch 80/100
109558/109558 [==============================] - 198s 2ms/step - loss: 1.9075 - acc: 0.5598
Epoch 81/100
109558/109558 [==============================] - 184s 2ms/step - loss: 1.8838 - acc: 0.5649
Epoch 82/100
109558/109558 [==============================] - 186s 2ms/step - loss: 1.8570 - acc: 0.5700
Epoch 83/100
109558/109558 [==============================] - 191s 2ms/step - loss: 1.8340 - acc: 0.5737
Epoch 84/100
109558/109558 [==============================] - 204s 2ms/step - loss: 1.8086 - acc: 0.5799
Epoch 85/100
109558/109558 [==============================] - 206s 2

# Save Model

In [12]:
# save the model to file
model.save('model/livros_model.h5')
# save the tokenizer
dump(tokenizer, open('model/livros_tokenizer.pkl', 'wb'))
#save 